In [7]:
import time
import pandas as pd
from census import Census
import altair as alt
import numpy as np

# Census API access
api_key = "639f2aedf7c17b164527591258cda00b25249b4b"
c = Census(key=api_key)

In [2]:
# Occupancy: Table B25002
occupy_variables = {
    'B25002_001E': 'total',
    'B25002_001M': 'total_moe',
    'B25002_002E': 'occupied',
    'B25002_002M': 'occupied_moe',
    'B25002_003E': 'vacant',
    'B25002_003M': 'vacant_moe'
}
occupy_columns_out = [
    'total', 'total_moe',
    'pct_occupied', 'pct_occupied_moe',
    'pct_vacant', 'pct_vacant_moe'
]

In [1]:
def combine_occupancy(in_df):
    '''
    Outputs: simplified vehicle breakdown with MOEs
    '''
    df = in_df.copy()
    
    for group in ['occupied', 'vacant']:
        # Calculate the proportion for this group
        df[f'pct_{group}'] = df[group] / df['total']
    
        # Calculate the MOE for this proportion
        df[f'pct_{group}_moe'] = (df[f'{group}_moe']**2 - df[f'pct_{group}']**2 * df['total_moe']**2)**0.5 / df['total']

        #NaN-out any too-low absolute n
        df.loc[df['total'] < 25, f'pct_{group}'] = float('NaN')
        df.loc[df['total'] < 25, f'pct_{group}_moe'] = float('NaN')
        
        #NaN-out any too-low moe
        df[f'pct_{group}_moe_ratio'] = df[f'pct_{group}_moe']/df[f'pct_{group}']
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}'] = float('NaN')
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}_moe'] = float('NaN')
    
    return df

In [3]:
def combine_tracts(in_df):
    '''

    Outputs:
    A modified version of in_df with census tracts combined
    '''
    
    # Start by making a copy of in_df, so we don't destroy the original data.
    # df is also a nice short name we can use throughout this function
    df_copy = in_df.copy()
    df = pd.DataFrame(columns=occupy_variables.values())

    ### CLEAN UNUSUAL MOES
    df_copy = df_copy.replace(-555555555.0, 0)

    for c in occupy_variables.values():
        if "moe" not in c:
            # sum the totals
            df.at[0, c] = df_copy[c].sum()
        else:
            # sum of squares the moes
            df.at[0, c] = np.sqrt((df_copy[c]**2).sum())

    df_out = combine_occupancy(df)
    
    return df_out

In [4]:
def get_occupy_precombo(year_in, place_num):
    # for getting mode for census tracts before they are combined
    df = pd.DataFrame(
        c.acs5.get(
            list(occupy_variables.keys()),
            {'for': place_num, 'in': 'state:06 county:013'},
            year=year_in
        )
    )
    df = df.rename(columns=occupy_variables)
    df = df.drop(columns=["state", "county", "tract"])
    
    return df

In [5]:
def get_city_df(year_in):
    # City
    df = pd.DataFrame(
        c.acs5.get(
            list(occupy_variables.keys()),
            {'for': 'place:60620', 'in': 'state:06'},
            year=year_in
        )
    )
    df = df.rename(columns=occupy_variables)
    df_out = combine_occupancy(df)

    return df_out

In [11]:
# for 5yr ACS 2014 and 2019
# Get ACS Table B08141 for select tracts around the BART station
# county:013
# city: 60620
c_tracts = 'tract:375000, 376000, 377000, 374000, 381000'

df_tracts_2014 = get_occupy_precombo(2014, c_tracts)
df_tracts_2019 = get_occupy_precombo(2019, c_tracts)

In [9]:
df_tracts_2014

,total,total_moe,occupied,occupied_moe,vacant,vacant_moe
0,1256.0,61.0,1093.0,102.0,163.0,89.0
1,2084.0,65.0,1849.0,110.0,235.0,104.0
2,2460.0,71.0,2160.0,143.0,300.0,130.0
3,1882.0,53.0,1714.0,118.0,168.0,109.0
4,2209.0,64.0,2050.0,101.0,159.0,95.0


In [14]:
df_comb_2014 = combine_tracts(df_tracts_2014)
df_tracts_out_2014 = df_comb_2014[occupy_columns_out]
df_tracts_out_2014.insert(0, "year", 2014)
df_comb_2019 = combine_tracts(df_tracts_2019)
df_tracts_out_2019 = df_comb_2019[occupy_columns_out]
df_tracts_out_2019.insert(0, "year", 2019)

In [15]:
df_tracts_out_2014

,year,total,total_moe,pct_occupied,pct_occupied_moe,pct_vacant,pct_vacant_moe
0,2014,9891.0,141.039002,0.89637,0.022853,0.10363,0.023995


In [16]:
# for 5yr ACS 2014 and 2019
# Get ACS Table B08141 in Richmond City
df_city_2014 = get_city_df(2014)
df_city_2019 = get_city_df(2019)

In [17]:
df_city_out_2014 = df_city_2014[occupy_columns_out]
df_city_out_2014.insert(0, "year", 2014)
df_city_out_2019 = df_city_2019[occupy_columns_out]
df_city_out_2019.insert(0, "year", 2019)

In [18]:
df_city_out_2014

,year,total,total_moe,pct_occupied,pct_occupied_moe,pct_vacant,pct_vacant_moe
0,2014,39772.0,736.0,0.915544,0.007751,0.084456,0.009706


In [19]:
#export settings, for a combined csv
df_tracts_out = pd.concat([df_tracts_out_2014, df_tracts_out_2019])
df_city_out = pd.concat([df_city_out_2014, df_city_out_2019])

In [20]:
df_tracts_out

,year,total,total_moe,pct_occupied,pct_occupied_moe,pct_vacant,pct_vacant_moe
0,2014,9891.0,141.039002,0.89637,0.022853,0.10363,0.023995
0,2019,9806.0,159.831161,0.908831,0.018661,0.091169,0.019392


In [21]:
df_city_out

,year,total,total_moe,pct_occupied,pct_occupied_moe,pct_vacant,pct_vacant_moe
0,2014,39772.0,736.0,0.915544,0.007751,0.084456,0.009706
0,2019,39643.0,848.0,0.935550,NaN,0.064450,0.009511
